### Use case description for the CCB cadence python package ###

In [8]:
import os
import glob
import numpy as np
import pandas as pd
from pathlib import Path
import time
import timeboard as tb
import timeboard.calendars.US as US
import datetime
import holidays
from faker import Faker

# Appearance of the Notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Import the package
%load_ext autoreload
%autoreload 2
import cadence
from cadence.mscheduler import Meetings
print(f'Package version: {cadence.__version__}')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Package version: 0.0.post1.dev5+g0626a2e


### Meeting participants and settings ###

In [27]:
# Create a list of random names
n_participants = 9

# Random seed
seed = 123

# Create fake data
fake = Faker()
Faker.seed(seed)
name_list = [fake.name() for name in range(n_participants)]
print(name_list)

# Day of the week we want to have a meeting
meeting_day_str = 'Wednesday'

# This translates to a number that the timeboard package needs
days_str = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
days_dict = dict(zip(days_str, range(len(days_str))))
display(days_dict)

['Brandon Russell', 'Steven Johnson', 'Evelyn Christian', 'George Cook', 'Aaron Graham', 'Kyle Jones', 'Jerome Whitehead', 'Charles Tyler', 'Thomas Berry']


{'Monday': 0,
 'Tuesday': 1,
 'Wednesday': 2,
 'Thursday': 3,
 'Friday': 4,
 'Saturday': 5,
 'Sunday': 6}

In [26]:
# Create a data frame with the names
name_df = pd.DataFrame({'full_name': name_list})
name_df[['name', 'last_name']] = name_df['full_name'].str.split(' ', expand=True)
display(name_df)

,full_name,name,last_name
0,Brandon Russell,Brandon,Russell
1,Steven Johnson,Steven,Johnson
2,Evelyn Christian,Evelyn,Christian
3,George Cook,George,Cook
4,Aaron Graham,Aaron,Graham
5,Kyle Jones,Kyle,Jones
6,Jerome Whitehead,Jerome,Whitehead
7,Charles Tyler,Charles,Tyler
8,Thomas Berry,Thomas,Berry


### Create a sequence of presenters ###

In [28]:
# Get the list of presenters from the data frame in alphabetical order
name_col = 'name'
presenters = sorted(list(name_df[name_col].unique()))
print(presenters)

['Aaron', 'Brandon', 'Charles', 'Evelyn', 'George', 'Jerome', 'Kyle', 'Steven', 'Thomas']


In [31]:
# If we are not happy with this sequence, we can rotate it
# Or, just change it manually
from cadence.mscheduler import cyclic_permutate
old_presenters = presenters.copy()
# Rotate the list, let's say to start with one particular name
new_presenters = cyclic_permutate(lst_in=old_presenters, name='Evelyn')
print(old_presenters)
print(new_presenters)

['Aaron', 'Brandon', 'Charles', 'Evelyn', 'George', 'Jerome', 'Kyle', 'Steven', 'Thomas']
['Evelyn', 'George', 'Jerome', 'Kyle', 'Steven', 'Thomas', 'Aaron', 'Brandon', 'Charles']


### Create a sequence of weekly meetings ###

In [4]:
# Lets do some cleanup of the spreadsheet
# small letters for the column names
tm = tm_raw.copy()
tm.columns = [col.lower() for col in tm.columns]

# small letters for emails and group, first letter capitalize for names
tm[email_col] = tm[email_col].str.lower()
tm[group_col] = tm[group_col].str.lower()
tm[name_col] = tm[name_col].str.title()

# Sort by group and then name
tm = tm.sort_values(by=['group', 'name'], ascending=True).reset_index(drop=True)

# save the name ist as a csv_file
presenter_list_file_name = f'{os.path.splitext(group_members_file_name)[0]}_list.csv'
presenter_list_file = os.path.join(data_dir, presenter_list_file_name)
tm.to_csv(presenter_list_file, index=False)
display(tm)

,name,email,group
0,Andreas,andreas_werdich@hms.harvard.edu,ai-data
1,Gerald,gerald_kiwanuka@hms.harvard.edu,ai-data
2,Nathan,nathan_palmer@hms.harvard.edu,ai-data
3,Sam,samantha_pullman@hms.harvard.edu,ai-data
4,Sreya,sreya_banerjee@hms.harvard.edu,ai-data
5,Andrew,andrew_ghazi@hms.harvard.edu,comp-bio
6,Anthony,anthony-alexander_christidis@hms.harvard.edu,comp-bio
7,Grey,grey_kuling@hms.harvard.edu,comp-bio
8,Ludwig,ludwig_geistlinger@hms.harvard.edu,comp-bio
9,Tram,tram_nguyen@hms.harvard.edu,comp-bio


In [5]:
# People who are not presenting
rm_name = ['nathan', 'ludwig']
rm_name = [nm.title() for nm in rm_name]

# Remove the names that we do not want in the schedule
presenters_df = tm.loc[~tm.get(name_col).isin(rm_name)]

In [6]:
# Merge two lists alternatively. 
# Start with shuffling everyone
presenters_df = presenters_df.sample(frac=1, random_state=123)
# And this method does not care if the lists do not have the same length.
group_list = presenters_df.get('group').unique()
name_lists = [sorted(list(presenters_df.loc[presenters_df.get(group_col) == grp, name_col].values)) \
           for grp in group_list]
presenters = merge_alternatively(name_lists[0], name_lists[1])
print(presenters)

['Andrew', 'Andreas', 'Anthony', 'Gerald', 'Grey', 'Sam', 'Tram', 'Sreya', 'Tyrone']


In [7]:
# Make the calendar
def create_timeboard(start_date, 
                     end_date, 
                     name_list, 
                     start_name, 
                     meeting_day=2,
                     date_col=date_col,
                     name_col=name_col,
                     comment_col=comment_col):

    # Create a holiday calendar
    us_ma_holidays = holidays.country_holidays('US', subdiv='MA')

    # Rotate the speakers so that the start_name comes first
    nlist = cyclic_permutate(name_list, start_name)
    
    # Define the list of speakers
    team_order = tb.RememberingPattern(nlist)
    
    # Set a weekly marker for every Wednesday
    week_day = tb.Marker(each='W', at=[{'days': meeting_day}])
    weekly = tb.Organizer(marker=week_day, structure=team_order)
    cal = tb.Timeboard(base_unit_freq='D', start=start_date, end=end_date, layout=weekly)
    cal = cal.to_dataframe()
    cal = cal.reset_index(drop=True)[['start', 'label']].\
                    rename(columns={'start': date_col, 'label': name_col})

    # Merge with the other member information
    cal = cal.merge(right=tm, on='name', how='left')
    
    # Add the MA holidays to this data frame
    cal = cal.assign(holiday=cal.get('date').apply(lambda dt: dt in us_ma_holidays),
                     comment=cal.get('date').apply(lambda dt: us_ma_holidays.get(dt)))
    
    return cal

In [18]:
# Create a new meeting schedule
start_date = ('2024-08-14')
end_date = ('2025-03-24')
cal_df = create_timeboard(start_date=start_date, 
                          end_date=end_date, 
                          name_list=presenters,
                          start_name='andreas')

print(presenters)
display(cal_df)

['Andrew', 'Andreas', 'Anthony', 'Gerald', 'Grey', 'Sam', 'Tram', 'Sreya', 'Tyrone']


/home/andreas/gitrepos/pyutils/.venv/lib/python3.10/site-packages/timeboard/when.py:37: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  start_times = pi.to_timestamp(how='start', freq='S')


,date,name,email,group,holiday,comment
0,2024-08-14,Andreas,andreas_werdich@hms.harvard.edu,ai-data,False,None
1,2024-08-21,Anthony,anthony-alexander_christidis@hms.harvard.edu,comp-bio,False,None
2,2024-08-28,Gerald,gerald_kiwanuka@hms.harvard.edu,ai-data,False,None
3,2024-09-04,Grey,grey_kuling@hms.harvard.edu,comp-bio,False,None
4,2024-09-11,Sam,samantha_pullman@hms.harvard.edu,ai-data,False,None
5,2024-09-18,Tram,tram_nguyen@hms.harvard.edu,comp-bio,False,None
6,2024-09-25,Sreya,sreya_banerjee@hms.harvard.edu,ai-data,False,None
7,2024-10-02,Tyrone,tyrone_lee@hms.harvard.edu,comp-bio,False,None
8,2024-10-09,Andrew,andrew_ghazi@hms.harvard.edu,comp-bio,False,None
9,2024-10-16,Andreas,andreas_werdich@hms.harvard.edu,ai-data,False,None


### Function to skip dates ###

In [19]:
def skip_date(cal_df, skip_date, skip_comment, name_col='name', date_col='date', comment_col='comment'):
    """ Skip dates in timeboard output """
    cdf_skip = cal_df.loc[cal_df.get(date_col)==skip_date]
    cdf_skip.loc[cdf_skip.get(date_col)==skip_date, comment_col] = skip_comment
    if len(cdf_skip)>0:
        cal_df_before = cal_df.loc[cal_df.get(date_col) < skip_date]
        cal_df_after = cal_df.loc[cal_df.get(date_col) > skip_date]
        name_list = list(cal_df.get(name_col).unique())
        # Re-create the calendar for the dates after the skip date
        cal_df_after = create_timeboard(start_date=cal_df_after[date_col].min(), 
                                        end_date=cal_df_after[date_col].max(), 
                                        name_list=name_list,
                                        start_name=cdf_skip.get('name').values[0])
        cal_df = pd.concat([cdf_before, cdf_skip, cdf_after], axis=0, ignore_index=True)
    else:
        raise ValueError(f'Skip date {skip_date} is not in data.')
    return cal_df

In [20]:
# Test this function
cal_df = skip_date(cal_df=cal_df,
                   skip_date='2024-09-04',
                   skip_comment='No meeting, party time.')

display(cal_df)

/home/andreas/gitrepos/pyutils/.venv/lib/python3.10/site-packages/timeboard/when.py:37: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  start_times = pi.to_timestamp(how='start', freq='S')


,date,name,email,group,holiday,comment
0,2024-08-14,Andreas,andreas_werdich@hms.harvard.edu,ai-data,False,None
1,2024-08-21,Anthony,anthony-alexander_christidis@hms.harvard.edu,comp-bio,False,None
2,2024-08-28,Gerald,gerald_kiwanuka@hms.harvard.edu,ai-data,False,None
3,2024-09-04,Grey,grey_kuling@hms.harvard.edu,comp-bio,False,"No meeting, party time."
4,2024-09-11,Grey,grey_kuling@hms.harvard.edu,comp-bio,False,None
5,2024-09-18,Sam,samantha_pullman@hms.harvard.edu,ai-data,False,None
6,2024-09-25,Tram,tram_nguyen@hms.harvard.edu,comp-bio,False,None
7,2024-10-02,Sreya,sreya_banerjee@hms.harvard.edu,ai-data,False,None
8,2024-10-09,Tyrone,tyrone_lee@hms.harvard.edu,comp-bio,False,None
9,2024-10-16,Andrew,andrew_ghazi@hms.harvard.edu,comp-bio,False,None


### Function to swap meeting dates ###

In [66]:
def in_list(lst, elements):
    """ Return true only if all elements are in the list lst """
    check_lst = [True if el in lst else False for el in elements]
    return all(check_lst)

def swap_elements(lst, el1, el2):
    """ Swap el1 and el2 in list lst if both el1 and el2 are in lst """
    new_lst = lst.copy()
    if in_list(lst=lst, elements=[el1, el2]):
        idx1, idx2 = lst.index(el1), lst.index(el2)
        new_lst[idx1], new_lst[idx2] = new_lst[idx2], new_lst[idx1]
    else:
        raise ValueError(f'Input list "lst" does not contain "{el1}" or "{el2}".')
    return new_lst

In [70]:
lst = ['a', 'b', 'c', 'd', 'e', 'f']
el_lst = ['b', 'c']
print(in_list(lst=lst, elements=el_lst))
print(lst)
print(swap_elements(lst=lst, el1='b', el2='x'))

True
['a', 'b', 'c', 'd', 'e', 'f']


ValueError: Input list "lst" does not contain "b" or "x".

In [62]:
'x' in lst

False

In [23]:
# Create a new meeting schedule
start_date = '2024-08-14'
end_date = '2025-01-31'

cal_df = create_timeboard(start_date=start_date, 
                          end_date=end_date, 
                          name_list=presenters,
                          start_name='anthony')

display(cal_df)

/home/andreas/gitrepos/pyutils/.venv/lib/python3.10/site-packages/timeboard/when.py:37: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  start_times = pi.to_timestamp(how='start', freq='S')


,date,name,email,group,holiday,comment
0,2024-08-14,Anthony,anthony-alexander_christidis@hms.harvard.edu,comp-bio,False,None
1,2024-08-21,Gerald,gerald_kiwanuka@hms.harvard.edu,ai-data,False,None
2,2024-08-28,Grey,grey_kuling@hms.harvard.edu,comp-bio,False,None
3,2024-09-04,Sam,samantha_pullman@hms.harvard.edu,ai-data,False,None
4,2024-09-11,Tram,tram_nguyen@hms.harvard.edu,comp-bio,False,None
5,2024-09-18,Sreya,sreya_banerjee@hms.harvard.edu,ai-data,False,None
6,2024-09-25,Tyrone,tyrone_lee@hms.harvard.edu,comp-bio,False,None
7,2024-10-02,Andrew,andrew_ghazi@hms.harvard.edu,comp-bio,False,None
8,2024-10-09,Andreas,andreas_werdich@hms.harvard.edu,ai-data,False,None
9,2024-10-16,Anthony,anthony-alexander_christidis@hms.harvard.edu,comp-bio,False,None


In [27]:
# Swap two dates
dt1 = '2024-08-28' # Grey
dt2 = '2024-09-18' # Sreya

cal_df_before = cal_df.copy()
cal_df_after = cal_df.copy()

date_list = cal_df.get(date_col).values
cal_df_after.index = date_list



In [28]:
display(cal_df_after.head(7))

,date,name,email,group,holiday,comment
2024-08-14,2024-08-14,Anthony,anthony-alexander_christidis@hms.harvard.edu,comp-bio,False,None
2024-08-21,2024-08-21,Gerald,gerald_kiwanuka@hms.harvard.edu,ai-data,False,None
2024-08-28,2024-08-28,Grey,grey_kuling@hms.harvard.edu,comp-bio,False,None
2024-09-04,2024-09-04,Sam,samantha_pullman@hms.harvard.edu,ai-data,False,None
2024-09-11,2024-09-11,Tram,tram_nguyen@hms.harvard.edu,comp-bio,False,None
2024-09-18,2024-09-18,Sreya,sreya_banerjee@hms.harvard.edu,ai-data,False,None
2024-09-25,2024-09-25,Tyrone,tyrone_lee@hms.harvard.edu,comp-bio,False,None


In [14]:
display(cal_new)

,date,name,email,group,holiday,comment
date,,,,,,
2024-08-28,2024-08-28,Gerald,gerald_kiwanuka@hms.harvard.edu,ai-data,False,None
2024-09-18,2024-09-18,Sam,samantha_pullman@hms.harvard.edu,ai-data,False,None
